In [1]:
cd /data5/galaxy/project/promoter_new_version/data/total/hand_work/2kb_2kb

/data5/galaxy/project/promoter_new_version/data/total/hand_work/2kb_2kb


In [2]:
import os
import glob
import pandas as pd
import numpy as np
from collections import Counter
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
# based on the defination of promoter region, each promoter must overlap with its according gene body.This would be kept later.
# at the same time, each promoter region maybe overlap with others genes. This would be throw later.

In [39]:
!bedtools merge -i promoter_2k-2k.bed | bedtools intersect -a promoter_2k-2k.bed -b stdin -f 1 -r -wa | sort -k1,1 -k2,2n | uniq > common.bed
!bedtools intersect -a common.bed -b /data/database/GRCh38/GENCODE/Genes_ensembl.bed -wa > overlap.bed

In [6]:
!wc -l common.bed

38749 common.bed


In [40]:
df = pd.read_table("overlap.bed", sep="\t", header=None, names=["chr", "start", "end", "gene id", "score", "strand"])
df["index_id"] = df["chr"] + "-" + df["gene id"]
df.head()

,chr,start,end,gene id,score,strand,index_id
0,chr1,9869,13869,ENSG00000223972,4000,+,chr1-ENSG00000223972
1,chr1,15436,19436,ENSG00000278267,4000,-,chr1-ENSG00000278267
2,chr1,15436,19436,ENSG00000278267,4000,-,chr1-ENSG00000278267
3,chr1,34081,38081,ENSG00000237613,4000,-,chr1-ENSG00000237613
4,chr1,50473,54473,ENSG00000268020,4000,+,chr1-ENSG00000268020


In [ ]:
# pick out the promoter regiones which only overlapped with self-gene.(overlap num == 1)

In [41]:
gene_list = df["index_id"].tolist()
# gene_list.count("chrY-ENSG00000228572")
# df[df["index_id"] == "chrY-ENSG00000228572"]
count = Counter(gene_list)
uniq_gene_list = [x for x, num in count.items() if num == 1]
len(uniq_gene_list)

22712

In [42]:
df["boo"] = np.where((df["chr"] + "-" + df["gene id"]).isin(uniq_gene_list), "True", "False")
df_uniq = df[df["boo"] == "True"][["chr", "start", "end", "gene id", "score", "strand"]].drop_duplicates()
df_uniq.to_csv("clean_promoter.bed", sep="\t", header=None, index=False)
df_uniq.head()

,chr,start,end,gene id,score,strand
0,chr1,9869,13869,ENSG00000223972,4000,+
3,chr1,34081,38081,ENSG00000237613,4000,-
4,chr1,50473,54473,ENSG00000268020,4000,+
5,chr1,55598,59598,ENSG00000240361,4000,+
10,chr1,171862,175862,ENSG00000241860,4000,-


In [43]:
len(df_uniq)

22712